In [1]:
import abc
import ollama

class Agent(abc.ABC):
    def __init__(self):
        self.sys_message = {'role': 'system', 'content':self.system_prompt()}
        self.user_message_default = "Introduce yourself."

    @abc.abstractmethod
    def system_prompt(self):
        """Return the system prompt content"""
        pass
    
    @abc.abstractmethod
    def user_prompt(self):
        pass

    def chat(self, user_input=None, model="gemma3n"):

        if user_input:
            user_content = self.user_prompt(user_input)
        else:
            user_content = self.user_message_default

        user_message = {'role': 'user', 'content': user_content}

        response = ollama.chat(model=model,messages=[self.sys_message, user_message])
        return response['message']['content'].strip()


In [2]:
class MetaAgent(Agent):
    def system_prompt(self):
        return """You are an expert machine learning researcher testing agentic systems for machine translation. 
        Your objective is to design building blocks such as prompts and workflows within these systems to solve complex task.
        Your aim is to design an optimal agent performing well on translation."""
    
    def user_prompt(self, user_input):
        return f"""You are deeply familiar with prompting techniques and the agent works from the literature. 
        Your goal is to maximized performance by proposing interestingly new agents. 
        Observe the discovered agents carefully and think about what insights, lessons, or stepping stones can be learned from them.
        
        The best performing agents are:

        {user_input}

        Propose the next interesting agentic system design. 
        Your first key should be (system_prompt) and it should be an improved system prompt for the translation agent.
        Your second key should be (user_prompt) and it should be an improved user prompt for the agent.
        Provide no other output or context.

        Response Template:
            {{'system_prompt': [your suggested prompt], 'user_prompt': [your suggested user prompt]}}
        """

In [3]:
import json
import json

def load_top_agents_from_file(filepath):
    """
    Load agent definitions from a JSON file.

    If 3 or fewer agents exist, all are returned.
    If more than 3, only the 3 with the highest 'score' are returned.

    Also returns the highest agent 'id' found in the file.

    Args:
        filepath (str): Path to the JSON file.

    Returns:
        tuple:
            - List[dict]: Up to 3 agent definitions (dicts)
            - int: The highest agent ID in the file
    """
    with open(filepath, 'r', encoding='utf-8') as f:
        agents = json.load(f)

    if not isinstance(agents, list):
        raise ValueError("JSON file must contain a list of agents.")

    highest_id = max(agent.get("id", -1) for agent in agents)

    if len(agents) <= 3:
        return agents, agents, highest_id

    # Sort by 'score' descending and return top 3
    top_agents = sorted(agents, key=lambda x: x.get("score", 0), reverse=True)[:3]
    print(agents, top_agents, highest_id)
    return agents, top_agents, highest_id


In [4]:
filepath = "agents.json"
agents, top_agents, highest_id = load_top_agents_from_file(filepath)

In [5]:
class Translator(Agent):
    def __init__(self):
        self.system_message = None
        self.user_message_template = None

        super().__init__()
        
    def load_agent(self, json_data):
        # if json_data is a string, parse it; otherwise, assume it's already a dict
        if isinstance(json_data, str):
            agent_config = json.loads(json_data)["agent"]
        else:
            agent_config = json_data["agent"]

        # save agent attributes as instance variables
        self.system_message = agent_config["system_prompt"]
        self.user_message_template = agent_config["user_prompt"]
        
    def system_prompt(self):
        return self.system_message
    
    def user_prompt(self, user_input):
        return self.user_message_template + f'\n Source: {user_input}'

In [6]:
class Evaluator(Agent):
    def system_prompt(self):
        return "You are an expert in translation evaluation. You score translations based on fluency, literary quality, and meaning preservation."
    
    def user_prompt(self, user_input):
        return f"""Score the following translation on a continuous scale from 0 to 100, where score of zero means
                "no meaning preserved" and score of one hundred means "perfect meaning and grammar". 
                Source: {user_input['source']}
                Target: {user_input['target']}

                Output only a number with no additional explanation.
                """

In [7]:
tib = """སེམས་ཅན་ཐམས་ཅད་ཀྱི་སྡུག་བསྔལ་སེལ་བ་ལ་ཕྱག་འཚལ་ལོ། །
སེམས་ཅན་ཐམས་ཅད་སྡུག་བསྔལ་ལས་གྲོལ་བར་མཛད་པ་ལ་ཕྱག་འཚལ་ལོ། །
སེམས་ཅན་ཐམས་ཅད་ཀྱི་བཅིངས་པ་གཅོད་པ་ལ་ཕྱག་འཚལ་ལོ། །
སེམས་ཅན་ཐམས་ཅད་ལ་མི་འཇིགས་པ་སྦྱིན་པ་ལ་ཕྱག་འཚལ་ལོ། །"""

eng = """Homage to the one who dispels the suffering of all beings.
Homage to the one who liberates all beings from suffering.
Homage to the one who cuts the bonds of all beings.
Homage to the one who grants fearlessness to all beings."""

In [8]:
import re
import json

translator = Translator()
evaluator = Evaluator()
meta = MetaAgent()

def score_agent(agent, translator=translator, evaluator=evaluator):

    translator.load_agent(agent)
    translation = translator.chat(tib)
    agent['translation'] = translation
    score = evaluator.chat({'source': tib, 'target':translation})
    agent['score'] = score

    return agent

def the_loop(translator=translator, evaluator=evaluator, meta=meta):

    filepath = "new-agents.json" 
    agents, top_agents, highest_id = load_top_agents_from_file(filepath)

    for agent in agents:
        if agent['score'] == "n/a":
            agent = score_agent(agent)

    success = False

    while success == False:
        try:
            suggested_agent = meta.chat(top_agents)
            cleaned = re.sub(r'^```json\s*|\s*```$', '', suggested_agent.strip())
            cleaned = cleaned.replace("'", '"')
            parsed = json.loads(cleaned)

            highest_id += 1

            new_agent = {"id": highest_id,
                        "agent": parsed,
                        "score": 0}
            
            new_agent = score_agent(new_agent)

            success = True
        except:
            success = False

    agents.append(new_agent)

    with open('new-agents.json', 'w', encoding='utf-8') as f:
        json.dump(agents, f, indent=2, ensure_ascii=False)


In [9]:
for _ in range(4):
    the_loop(translator=translator, evaluator=evaluator, meta=meta)

[{'id': 0, 'agent': {'system_prompt': 'You are an expert translator of Tibetan Buddhist texts. You translate clearly and concisely, preserving the original intent of the text.', 'user_prompt': 'With no additional output or context, translate the following from Tibetan to English:{SRC}'}, 'score': '85', 'translation': 'I pay homage to all sentient beings for alleviating their suffering.\nI pay homage to you for liberating all sentient beings from suffering.\nI pay homage to you for cutting through the bonds of all sentient beings.\nI pay homage to you for granting fearlessness to all sentient beings.'}, {'id': 1, 'agent': {'system_prompt': 'You are a highly skilled and culturally sensitive machine translator specializing in nuanced translations, particularly of texts requiring preservation of philosophical and emotional intent. Prioritize accuracy, natural-sounding English, and avoid literal translations that sacrifice meaning. Consider the potential cultural implications of the source 